In [1]:
import numpy as np
import pandas as pd

In [2]:
col_names = {'weekend_nights_no', 'weekday_nights_no', 'lead_time', 'avg_price_per_room', 'special_requests_no', 'month', 'prev_show_rate', 'canceled'}
df = pd.read_csv("https://raw.githubusercontent.com/UIUC-DSC/Team-A/main/MLDataset.csv?token=GHSAT0AAAAAACBB4LJHLXHRWW6BVUERW7UMZCBUZNA")
df.head(10)

,long_trip,lead_time,avg_price_per_room,special_requests_no,month,prev_show_rate,canceled
0,1,224,65.00,0,10.0,0.5,0
1,1,5,106.68,1,11.0,0.5,0
2,1,1,60.00,0,2.0,0.5,1
3,1,211,100.00,0,5.0,0.5,1
4,1,48,94.50,0,4.0,0.5,1
5,1,346,115.00,1,9.0,0.5,1
6,1,34,107.55,1,10.0,0.5,0
7,1,83,105.61,1,12.0,0.5,0
8,1,121,96.90,1,7.0,0.5,0
9,1,44,133.44,3,10.0,0.5,0


In [3]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):

        '''constructor'''

        #for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        "for leaf node"
        self.value = value

In [4]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''

        #initialize root
        self.root = None

        #conditions to stop 
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        '''recursive function to build tree'''

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        # split tree until the stopping condition is met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            #find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            #Check if the info gain is positive
            if best_split["info_gain"]>0:
                #recursion to left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                #recursion to right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)

                #return decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        # compute the lead node    
        leaf_value = self.calculate_leaf_value(Y)
        #return the leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        '''function to find best split in tree'''
        

        #dictionary to store the best found split
        best_split = {}
        max_info_gain = -float("inf")

        #loop over all of the feature
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            #loop over all of the feature values present in all of the data 
            for threshold in possible_thresholds:

                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                #check for non-null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute info gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # if the statement is true best split will be updated
                    if curr_info_gain>max_info_gain :
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split
    
    def split(self, dataset, feature_index, threshold):
        '''function to split data'''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        '''function that computes info gain'''
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        '''function for calculating entropy'''

        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        '''function to find gini index'''

        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    
    def calculate_leaf_value(self, Y):
        '''compute leaf node'''
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        '''function to print the tree'''

        if not tree:
            tree=self.root
        
        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        
    def fit(self, X, Y):
        '''function to train tree'''

        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        '''function to predict the new dataset'''

        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    def make_prediction(self, x, tree):
        '''function to predict a single data point'''

        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

        

Train-Test Split

In [5]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

Fit the Model

In [6]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_1 <= 151.0 ? 0.0771420056827058
 left:X_3 <= 0.0 ? 0.021748533927452907
  left:X_2 <= 112.2 ? 0.0338527256910468
    left:X_1 <= 44.0 ? 0.02323635778496863
        left:0.0
        right:0.0
    right:X_1 <= 9.0 ? 0.04448052752946652
        left:0.0
        right:1.0
  right:X_3 <= 1.0 ? 0.004884002827521633
    left:X_1 <= 6.0 ? 0.005506133813801217
        left:0.0
        right:0.0
    right:X_1 <= 90.0 ? 0.007912264405514083
        left:0.0
        right:0.0
 right:X_2 <= 100.0 ? 0.08908033950976668
  left:X_3 <= 0.0 ? 0.055052141149536826
    left:X_2 <= 81.0 ? 0.016573569838355284
        left:1.0
        right:1.0
    right:X_1 <= 180.0 ? 0.026264204799394786
        left:0.0
        right:0.0
  right:X_4 <= 11.0 ? 0.04071720638407983
    left:X_3 <= 2.0 ? 0.02500607405271471
        left:1.0
        right:0.0
    right:X_3 <= 0.0 ? 0.19643360302700968
        left:0.0
        right:1.0


Test the Model

In [7]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred) 

0.7993108201240524